### This notebook shows how to write simple Prompt templates, custom Agent tools using Langchain and how Routing works

### 1. Without Prompt template

In [1]:
from langchain_core.runnables import RunnableLambda
from langchain_ollama import OllamaLLM


# Initialize the Ollama model (make sure it's running locally)
llm = OllamaLLM(model ="llama2") 

# Wrap into a Runnable using a simple lambda or chain
chain = RunnableLambda(lambda prompt: llm.invoke(prompt))

# Invoke the model
response = chain.invoke("What's the capital of Canada?")
print(response)


The capital of Canada is Ottawa.


### 2. 'With' Prompt template and passable parameters.

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create a prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

# Chain it
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
result = chain.invoke("Canada")
print(result)

/tmp/ipykernel_787516/2285740442.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'country': 'Canada', 'text': '\nThe capital of Canada is Ottawa.'}


In [3]:
# Run the chain
result = chain.invoke("India")
print(result)

{'country': 'India', 'text': '\nThe capital of India is New Delhi.'}


In [4]:
result = chain.invoke("US")
print(result)

{'country': 'US', 'text': '\nThe capital of the United States is Washington, D.C. (District of Columbia).'}


###  Define Agent Tool with Parameters

In [5]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.llms import Ollama

# Define the tool (manually)
def get_squared_func(number: str) -> str:
    """Takes a number as a string and returns its square. Handles messy input."""
    # Extract the first integer from the string
    import re
    match = re.search(r'\d+', number)
    if not match:
        return "Invalid input: No number found"
    num = int(match.group())
    return str(num ** 2)


get_squared_tool = Tool(
    name="get_squared",
    func=get_squared_func,
    description="Takes an number string as input and returns its square."
)

llm = Ollama(model="llama2")



/tmp/ipykernel_787516/496600779.py:22: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


### Set up the agent 

In [6]:
agent = initialize_agent(
    tools=[get_squared_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2, 
     early_stopping_method="generate"  # Optional: finish with best guess
)


/tmp/ipykernel_787516/1239551101.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### Test the tool

In [7]:
## # Now this prompt style is valid
# agent.run("What is the square of 12?")
# directly calling the tool

get_squared_tool.run('15')

'225'

### Building Agent tool as a Runnable pipeline

In [8]:
import re
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_community.llms import Ollama

# Step 1: Define the get_squared tool
def get_squared(number: int) -> int:
    return number * number

# Step 2: Parse numbers from input
def extract_numbers(text: str) -> list[int]:
    return [int(n) for n in re.findall(r'\d+', text)]

# Step 3: Build a LangChain Runnable pipeline
# - Input: user prompt
# - Output: formatted result string
pipeline = (
    RunnableLambda(lambda text: extract_numbers(text))  # ["12", "36"]
    | RunnableLambda(lambda nums: [(n, get_squared(n)) for n in nums])  # [(12, 144), (36, 1296)]
    | RunnableLambda(lambda results: ", ".join(f"{n}² = {sq}" for n, sq in results))  # "12² = 144, 36² = 1296"
)

# Optional: use Ollama to add LLM reasoning before/after
llm = Ollama(model="llama2")

# Run the pipeline
user_input = "What is the square of 12 and 36?"
result = pipeline.invoke(user_input)
print(result)


12² = 144, 36² = 1296


In [9]:
# Run the pipeline
user_input = "What is the square of 15?"
result = pipeline.invoke(user_input)
print(result)

15² = 225


### Explain the output using Prompting

In [10]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap

# Add an LLM explanation step
explanation_prompt = PromptTemplate.from_template(
    "Given these squared results: {results}, explain them in a friendly sentence."
)

chain = (
    pipeline
    | RunnableMap({"results": lambda x: x})  # name the result for the prompt
    | explanation_prompt
    | llm
)

print(chain.invoke(user_input))



 Hey there! So, you know how we've been learning about square numbers and stuff? Well, today we're gonna talk about something really cool - squares of numbers! That's right, I'm talking about when you take a number and multiply it by itself, like 15 x 15 = 225. It's like magic, right? You start with one number, and then you make it bigger and bigger just by multiplying it by itself! Isn't that awesome? 😍


The above example shows how to use 1 tool in a Langchain. Now we will explore how to use a router for LLM to pick from a bunch of tools

### How to implement a Router to pick tools: 

Below is a simple implementation of using two tools : Search and Calculator. And the LLM picks the tools based on the descriptions of the tool provided.

In [41]:
from langchain_community.chat_models import ChatOllama
from langchain.agents import Tool, initialize_agent, AgentType
import re


In [ ]:
# --- Tools ---
def search_tool_func(query: str) -> str:
    print ("Search tool was called")
    return f"Searching for {query}"

search_tool = Tool.from_function(
    func=search_tool_func,
    name="SearchTool",
    description="Useful for when you need to search the web."
)


def calculator_tool_func(expression: str) -> str:
    expression = expression.replace('x', '*').replace('X', '*')
    expression = re.sub(r'[^\d\+\-\*/\.\(\)]', '', expression)
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

calculator_tool = Tool.from_function(
    func=calculator_tool_func,
    name="CalculatorTool",
    description="Useful for solving math problems like '42 * 19'."
)

# --- LLM from Ollama ---
llm = ChatOllama(model="llama3")  # or "mistral", "llama2", etc.

# --- Agent ---
agent = initialize_agent(
    tools=[calculator_tool, search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3 # <-- Stop after 3 steps
)


# --- Run It ---
response = agent.run("What's 42 x 19?")
print("\n✅ Final Answer:", response)




> Entering new AgentExecutor chain...
A simple math problem! 

Thought: This one looks like a straightforward calculation.

Action: CalculatorTool
Action Input: "42 * 19"
Observation: 798
Thought:Here's my response:

Question: What's 42 x 19?

Thought: A simple math problem!

Thought: This one looks like a straightforward calculation.

Action: CalculatorTool
Action Input: "42 * 19"
Observation: 798
Thought:I think I've got the answer already!

Final Answer: The result of 42 x 19 is 798.

> Finished chain.

✅ Final Answer: The result of 42 x 19 is 798.


In [43]:
# --- Run It ---
response = agent.run("What's the capital of Canada?")
print("\n✅ Final Answer:", response)



> Entering new AgentExecutor chain...
Thought: Hmm, this seems like a non-math problem, so I won't be able to use the CalculatorTool. Let me think about what else I can do...

Action: SearchTool
Action Input: "What is the capital of Canada?"Search tool was called

Observation: Searching for What is the capital of Canada?
Thought:Here's my attempt:

Question: What's the capital of Canada?

Thought: Hmm, this seems like a non-math problem, so I won't be able to use the CalculatorTool. Let me think about what else I can do...

Action: SearchTool
Action Input: "What is the capital of Canada?"Search tool was called

Observation: Searching for What is the capital of Canada?
Thought: Ottawa!

Here's my attempt:

Question: What's the capital of Canada?

Thought: Hmm, this seems like a non-math problem, so I won't be able to use the CalculatorTool. Let me think about what else I can do...

Action: SearchTool
Action Input: "What is the capital of Canada?"Search tool was called

Observation: Se

In [4]:
from langchain_core.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, AgentType
#from langchain_openai import ChatOpenAI
import datetime

# 1. Define a custom tool (function)
@tool
def get_current_time(city: str) -> str:
    """Returns the current time in a given city."""
    # You can add actual timezone support with pytz/zoneinfo if needed
    now = datetime.datetime.now()
    return f"The current time in {city} is {now.strftime('%Y-%m-%d %H:%M:%S')}"

# 2. Initialize the language model
#llm = ChatOpenAI(model="gpt-4", temperature=0)
# --- LLM from Ollama ---
llm = ChatOllama(model="llama3")  # or "mistral", "llama2", etc.

# 3. Initialize the agent with the tool
agent = initialize_agent(
    tools=[get_current_time],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

# 4. Run the agent
response = agent.run("What is the exact current time in Tokyo? I am currently located in EST time zone")
print(response)




> Entering new AgentExecutor chain...
I can help you with that!

Tokyo is in Japan Standard Time (JST), which is UTC+9. To get the current time in Tokyo, we need to add 13 hours to your Eastern Standard Time (EST) since there's a 5-hour difference.

As I'm responding to you, it's:

* 12:00 PM EST (your local time)
* 1:00 AM JST (in Tokyo)

Please note that Japan does not observe daylight saving time, so the UTC offset remains constant throughout the year.

> Finished chain.
I can help you with that!

Tokyo is in Japan Standard Time (JST), which is UTC+9. To get the current time in Tokyo, we need to add 13 hours to your Eastern Standard Time (EST) since there's a 5-hour difference.

As I'm responding to you, it's:

* 12:00 PM EST (your local time)
* 1:00 AM JST (in Tokyo)

Please note that Japan does not observe daylight saving time, so the UTC offset remains constant throughout the year.


In [ ]:
# Adding pydantic